#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import Toolbox_ML as TB
import bootcampviztools as bt
import re
import os

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Data

In [2]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv ('.\desafio-de-prediccion-de-precios-de-portatiles/train.csv')
test = pd.read_csv ('.\desafio-de-prediccion-de-precios-de-portatiles/test.csv')

In [4]:
print (df_train.shape)
print (df_test.shape)

(912, 14)
(391, 13)


## Data exploration

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


En test solo falta la columna target "Price_euros"

In [6]:
df_train.describe()

,id,laptop_ID,Inches,Price_euros
count,912.000000,912.000000,912.000000,912.000000
mean,645.091009,666.192982,15.011404,1108.122873
std,380.313813,384.873846,1.411744,714.597741
min,0.000000,1.000000,10.100000,174.000000
25%,319.750000,330.500000,14.000000,589.000000
50%,636.500000,673.500000,15.600000,949.000000
75%,980.250000,998.500000,15.600000,1458.500000
max,1302.000000,1320.000000,18.400000,6099.000000


In [7]:
TB.describe_df(df_train)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
Tipos,int64,int64,object,object,object,float64,object,object,object,object,object,object,object,float64
% Faltante,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Valores Únicos,912,912,19,475,6,15,35,104,8,36,91,9,158,614
% Cardinalidad,100.0,100.0,2.08,52.08,0.66,1.64,3.84,11.4,0.88,3.95,9.98,0.99,17.32,67.32


In [8]:
TB.tipifica_variables (df_train, 10, 30)

,nombre_variable,tipo_sugerido
0,id,Numérica Continua
1,laptop_ID,Numérica Continua
2,Company,Numérica Discreta
3,Product,Numérica Continua
4,TypeName,Categórica
5,Inches,Numérica Discreta
6,ScreenResolution,Numérica Discreta
7,Cpu,Numérica Discreta
8,Ram,Categórica
9,Memory,Numérica Discreta


In [9]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


## Data processing

In [16]:
# Convertir contenido de columnas categóricas a minúsculas
categorical_cols = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Gpu', 'OpSys']

for col in categorical_cols:
    train[col] = train[col].astype(str).str.lower()
    test[col] = test[col].astype(str).str.lower()


In [19]:
# Procesar la columna 'Ram'
train['Ram'] = train['Ram'].astype(str).str.replace('gb', '').str.strip().astype(int)
test['Ram'] = test['Ram'].astype(str).str.replace('gb', '').str.strip().astype(int)

# Procesar la columna 'Weight'
train['Weight'] = train['Weight'].astype(str).str.replace('kg', '').str.strip().astype(float)
test['Weight'] = test['Weight'].astype(str).str.replace('kg', '').str.strip().astype(float)

# Procesar la columna 'ScreenResolution'
resolution_split_train = train['ScreenResolution'].astype(str).str.split('x', n=1, expand=True)
train['ScreenWidth'] = resolution_split_train[0].str.extract('(\d+)').astype(int)
train['ScreenHeight'] = resolution_split_train[1].str.extract('(\d+)').astype(int)

resolution_split_test = test['ScreenResolution'].astype(str).str.split('x', n=1, expand=True)
test['ScreenWidth'] = resolution_split_test[0].str.extract('(\d+)').astype(int)
test['ScreenHeight'] = resolution_split_test[1].str.extract('(\d+)').astype(int)

# Procesar la columna 'Memory'
def process_memory(memory):
    memory = memory.lower().replace('gb', '').replace('tb', '000')
    memory = re.sub("[^0-9]", "", memory)  # Filtrar solo los dígitos de la cadena
    if '+' in memory:
        parts = memory.split('+')
        hdd = int(parts[0].strip()) if 'hdd' in parts[0] else 0
        ssd = int(parts[1].strip()) if 'ssd' in parts[1] else 0
    else:
        hdd = int(memory.strip()) if 'hdd' in memory else 0
        ssd = int(memory.strip()) if 'ssd' in memory else 0
    return hdd, ssd

train['HDD'], train['SSD'] = zip(*train['Memory'].astype(str).map(process_memory))
test['HDD'], test['SSD'] = zip(*test['Memory'].astype(str).map(process_memory))

# Procesar la columna 'Cpu'
cpu_split_train = train['Cpu'].astype(str).str.split(' ', n=3, expand=True)
train['Cpu_Brand'] = cpu_split_train[0]
train['Cpu_Model'] = cpu_split_train[1]

cpu_split_test = test['Cpu'].astype(str).str.split(' ', n=3, expand=True)
test['Cpu_Brand'] = cpu_split_test[0]
test['Cpu_Model'] = cpu_split_test[1]

In [20]:
# Mostrar las primeras filas después de las transformaciones
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ScreenWidth,ScreenHeight,Cpu_Brand,Cpu_Model,HDD,SSD
0,268,1002,dell,inspiron 5567,notebook,15.6,1366x768,intel core i7 7500u 2.7ghz,8,1TB HDD,amd radeon r7 m445,windows 10,2.36,749.00,1366,768,intel,core,0,0
1,347,867,asus,x541na (n4200/4gb/1tb/w10),notebook,15.6,1366x768,intel pentium quad core n4200 1.1ghz,4,1TB HDD,intel hd graphics 505,windows 10,2.00,449.00,1366,768,intel,pentium,0,0
2,353,966,toshiba,portege z30-c-1cw,notebook,13.3,full hd 1920x1080,intel core i5 6200u 2.3ghz,8,256GB SSD,intel hd graphics 520,windows 7,1.20,1460.00,1920,1080,intel,core,0,0
3,578,767,dell,alienware 17,gaming,15.6,ips panel 4k ultra hd 3840x2160,intel core i7 7700hq 2.8ghz,16,256GB SSD + 1TB HDD,nvidia geforce gtx 1070,windows 10,4.42,2868.99,4,2160,intel,core,0,0
4,42,1241,dell,latitude e7270,ultrabook,12.5,full hd / touchscreen 1920x1080,intel core i5 6300u 2.4ghz,8,256GB SSD,intel hd graphics 520,windows 7,1.26,1713.37,1920,1080,intel,core,0,0


-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [39]:
# Definir X e y
X = train.drop(columns=['id', 'Price_euros', 'Product', 'Cpu', 'Memory', 'ScreenResolution'])
y = train['Price_euros']

# Convertir columnas categóricas a variables dummy
X = pd.get_dummies(X, drop_first=True)


### 2. Dividir X_train, X_test, y_train, y_test

In [40]:
# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar la forma de los conjuntos de datos
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((729, 142), (183, 142), (729,), (183,))

### 3. Crear y entrenar el modelo


In [41]:
# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# Definir la búsqueda de hiperparámetros para los modelos
param_grid = [
    {
        'model': [LinearRegression()],
    },
    {
        'model': [RandomForestRegressor()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
    },
    {
        'model': [LGBMRegressor()],
        'model__num_leaves': [31, 50],
        'model__n_estimators': [100, 200],
    },
    {
        'model': [XGBRegressor()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [3, 6, 9],
        'model__learning_rate': [0.01, 0.1, 0.3],
    }
]

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 29 candidates, totalling 87 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'model': [LinearRegression()]},
                         {'model': [RandomForestRegressor()],
                          'model__max_depth': [None, 10, 20],
                          'model__n_estimators': [100, 200]},
                         {'model': [LGBMRegressor()],
                          'model__n_estimators': [100, 200],
                          'model__num_leaves': [31, 50]},
                         {'mode...
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...)],
                          'model__learning_rate': [0.01, 0.1, 0.3],
                          'model__max_depth': [3, 6, 9],
                          'model__n_estimators': [100, 200]}],
             scoring='neg_mean_absolute_error', verbose=2)

In [42]:
# Mejor modelo
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


In [24]:
'''from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)
'''

LinearRegression()

In [43]:
'''from lightgbm import LGBMRegressor
model_lgbm = LGBMRegressor(random_state=42)
model_lgbm.fit(X_train, y_train)'''

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 445
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 35
[LightGBM] [Info] Start training from score 1122.043223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMRegressor(random_state=42)

In [45]:
'''from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train)'''

RandomForestRegressor(random_state=42)

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [43]:
# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE for the best model: {mae}')
print(f'Best parameters: {best_params}')

MAE for the best model: 188.92156280689514
Best parameters: {'model': RandomForestRegressor(), 'model__max_depth': 20, 'model__n_estimators': 100}


In [46]:
'''# Evaluar el modelo
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE Lineal Regresion: {mae}')'''

MAE Lineal Regresion: 252.62678947155825


In [47]:
'''y_pred_lgbm = model_lgbm.predict(X_test)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
print(f'MAE LightGBM: {mae_lgbm}')'''

MAE LightGBM: 209.11527077966758


In [48]:
'''y_pred_rf = model_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f'MAE Random Forest: {mae_rf}')'''

MAE Random Forest: 206.2213704918033


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [44]:
# Ya he preprocesado el conjunto de test de la misma manera que el conjunto de entrenamiento previamente

# Preprocesar el conjunto de test de la misma manera que el conjunto de entrenamiento
X_pred = test.drop(columns=['Product', 'Cpu', 'Memory', 'ScreenResolution'])
X_pred = pd.get_dummies(X_pred, drop_first=True)
X_pred = X_pred.reindex(columns=X.columns, fill_value=0)

### 2. Carga los datos de `test.csv` para predecir.

In [ ]:
# ya los cargué al principio

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [19]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [49]:
'''# Realizar predicciones Regresion lineal
predictions_lr = model.predict(X_pred)
#predictions

In [29]:
#predictions = model.predict(X_pred)
#predictions

In [50]:
'''# Realizar predicciones LGBM
predictions_lgbm = model_lgbm.predict(X_pred)'''

In [51]:
'''# Realizar predicciones Random Forest
predictions_rf = model_rf.predict(X_pred)'''

In [45]:
# Realizar predicciones con el mejor modelo
predictions = best_model.predict(X_pred)

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [46]:
sample = pd.read_csv(".\desafio-de-prediccion-de-precios-de-portatiles/sample_submission.csv")

In [34]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [35]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [52]:
'''# Crear el dataframe para la submission (Regresion Lineal)
submission_lr = sample.copy()
submission_lr['Price_euros'] = predictions_lr'''

In [53]:
'''submission_lgbm = sample.copy()
submission_lgbm['Price_euros'] = predictions_lgbm'''

In [54]:
'''submission_rf = sample.copy()
submission_rf['Price_euros'] = predictions_rf'''

In [53]:
# Crear el dataframe para la submission
#submission = pd.DataFrame({'id': test['id'], 'price_euros': predictions})
submission = sample.copy()
submission['Price_euros'] = predictions

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [54]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [28]:
#chequeator(submission_lr) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

In [51]:
train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros', 'ScreenWidth', 'ScreenHeight', 'Cpu_Brand', 'Cpu_Model',
       'HDD', 'SSD'],
      dtype='object')

In [52]:
test.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'ScreenWidth', 'ScreenHeight', 'Cpu_Brand', 'Cpu_Model', 'HDD', 'SSD'],
      dtype='object')

In [55]:
chequeator(submission)

You're ready to submit!


NameError: name 'urllib' is not defined

In [29]:
'''chequeator(submission_rf)'''

'chequeator(submission_rf)'

In [27]:
#chequeator(submission_lgbm)